In [2]:
# !sudo pip install --upgrade transformers
# !sudo pip install fuzzywuzzy

In [3]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import json
#from Gpt4_OpenAI_Prompt import system_msg,context,postprocess
#from prompt1 import system_msg
import pandas as pd
import transformers
from jinja2 import Template
from fuzzywuzzy import process
print(transformers.__version__)

4.34.1


/opt/conda/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
print(torch.cuda.is_available())
#print(torch.cuda.get_device_name())

False


In [5]:
context = {
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
                         "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
                        {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
                        {"ENTITY": "contribution_to_growth",
                         "other names": ["contribution to growth", "growth", "grown"]},
                        {"ENTITY": "kda_transactional",
                         "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
                        {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
                        {"ENTITY": "correlation",
                         "other names": ["associate", "associated", "association", "associations", "correlate",
                                         "correlated",
                                         "correlation", "correlations", "relate", "related", "relation", "relations",
                                         "relationship",
                                         "relationships"]}
                        ],
    "DATE VARIABLE": [
        {"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
}

In [6]:
def get_context_df():
    df_final = pd.DataFrame()
    for i in context:
        df = pd.DataFrame(context[i])
        df["Type"]  = i
        df_final = pd.concat([df,df_final],ignore_index=True)
    return df_final

In [ ]:
# ################################################################################
# # bitsandbytes parameters
# ################################################################################

# # Activate 4-bit precision base model loading
# use_4bit = True

# # Compute dtype for 4-bit base models
# bnb_4bit_compute_dtype = "float32"

# # Quantization type (fp4 or nf4)
# bnb_4bit_quant_type = "fp4"

# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# # Activate nested quantization for 4-bit base models (double quantization)
# use_nested_quant = False

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=use_4bit,
#     bnb_4bit_quant_type=bnb_4bit_quant_type,
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=use_nested_quant,
# )
# # Load the entire model on the GPU 0
# device_map = {"": 0}


In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
# model_mistral = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,
#     device_map=device_map)
model_mistral = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16)
tokenizer_mistral = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def gereneate_output_mistral(prompt):
    #prompt = json.dumps(prompt)
    model_input = tokenizer_mistral(prompt, return_tensors="pt")#.to("cuda")
    #model.eval()
    response = tokenizer_mistral.decode(model_mistral.generate(**model_input, max_length=1024*2)[0], skip_special_tokens=True)
    return response

In [9]:
def get_user_question_for_open_ai_prompt(question):
    #return """Now Based on this following "CONTEXT" data lookup:\n""" + str(context) + "\n return a response for the question:\n" + question + "\n"
    return "\n Now detect from this question:\n" + question + "\n"

def compose_prompt_open_ai_prompt(question,base_prompt):
    return f"""[INST]<<SYS>>
            {base_prompt}
            <</SYS>>
            {get_user_question_for_open_ai_prompt(question)}
            [/INST]
            [_separator_]"""

In [10]:
def find_best_match_from_context(word, df):
    matches = process.extractOne(word, df.values.flatten())
    max_match, score = matches
    index = df.values.flatten().tolist().index(max_match)
    row_index, col_index = divmod(index, df.shape[1])
    entity_name = df.iat[row_index, col_index]
    return df.iloc[row_index].to_dict()

In [11]:
MEASURE_LIST = sum([[i["ENTITY"]]+i["other names"] for i in context["MEASURE"]],[])
DIMENSION_LIST = sum([[i["ENTITY"]]+i["other names"] for i in context["DIMENSION"]],[])
FILTER_LIST = sum([[i["ENTITY"]]+i['other names'] for i in context["FILTER"]],[])

In [29]:
ADJECTIVE_and_TONE = f"""
Task: Extract Adjectives, Determine Intent (TONE), Return RANK ADJECTIVE, and Scalar (RANK)

Instructions:

1. Consider these list of dimension entities {DIMENSION_LIST} as DIMENSION_ENTITIES_LIST
2. For each dimension entity detected in the give question, proceed to determine if there is any Adjective intent present within the question, Adjectives can include terms like "least" "highest", etc
3. Determine the intent (TONE) of the adjective. The intent can be only be either positive or negative.
4. If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
5. If the adjective intent is positive, always set RANK ADJECTIVE to "TOP" and TONE to "positive."
6. If the adjective implies a quantity or magnitude (e.g., rank is 3 in "top 3"), extract that scalar value as "RANK"
7. Return the following attributes for each detected dimension entity in a JSON list format: """
output_format="""
[
{
"ADJECTIVE": only adjective whichever is detected else null,
"TONE": "positive" or "negative"
"ENTITY": The specific dimension entity matched from the DIMENSION_ENTITIES_LIST.,
"RANK ADJECTIVE": "TOP" or "BOTTOM",
"RANK": "if value detected else null"
}
]
"""
ADJECTIVE_and_TONE = ADJECTIVE_and_TONE+output_format


In [12]:


# ADJECTIVE_and_TONE = """
# **Task**: Extract Adjectives, Determine Intent (TONE), Identify Entity, Return RANK ADJECTIVE, and Scalar (RANK)

# **Instructions**:

# 1. Carefully read the user question to identify any adjectives present. Adjectives can include terms like "least," "highest performing," etc.
# 2. Determine the intent (TONE) of the adjective. The intent can be either positive or negative.
#    - If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
#    - If the adjective intent is positive, always  set RANK ADJECTIVE to "TOP" and TONE to "positive."
# 3. If the adjective implies a quantity (e.g., "how many," "top 3"), extract that scalar value as RANK.
# 4. Identify the entity (e.g., "Segment") on which the detected adjective should be applied.

# **Input Question**: "How many top 3 segments are performing well?"

# **Output**:
# {
# "ADJECTIVE": "performing",
# "TONE": "positive",
# "ENTITY": "Segment",
# "RANK ADJECTIVE": "TOP",
# "TONE": "positive",
# "RANK": "3"
# }

# **Note**: If the input question doesn't contain an adjective, doesn't imply a quantity, or if it's not clear which entity the adjective should be applied to, you can indicate that they were not detected.
# """

In [16]:
##*DONE*##

FILTER_PROMPT = f'''
Task: Extract Filter Entities
1. Consider these list of filter entities {FILTER_LIST} as FILTER_ENTITIES_LIST.
2. Analyze the provided question or text.
3. For each filter entity detected in the question or text, determine if it's intended to be included or excluded.
4. Return the following attributes for each detected filter entity:'''
output_format = """
{
"EXCLUDE": (Set to TRUE if the entity is intended to be excluded, FALSE otherwise.),
"ENTITY": (The specific filter entity matched from the FILTER_ENTITIES_LIST.)
}
If the question specifies exclusion for an entity, set EXCLUDE to TRUE; otherwise, set it to FALSE."""
FILTER_PROMPT = FILTER_PROMPT+output_format

In [17]:
##*DONE*##

MEASURE_CONSTRAINT = f'''
Task: Extract metric constraints and Measure Entities
1. Consider these list of measure entities {MEASURE_LIST} as MEASURE_ENTITIES_LIST.
2. For each measure entity detected in the give question, proceed to determine if there is any comparison intent present within the question, such as "greater than," "less than," or "equals to."
3. For all such only matched measure entities extract the comparison operator (such as >,<,<=,>=, =) as "COMPARISON OPERATOR" (if a comparison intent is present else null).
4. For all such only matched measure entities extract the "COMPARISON VALUE" which should be a numerical or scalar value which will be operationg on "COMPARISON OPERATOR".
5. Return the following attributes for each detected measure entity in a JSON list format:'''

output_format = """
[
{
"ENTITY": (The specific measure entity matched from the MEASURE_ENTITIES_LIST.),
"COMPARISON OPERATOR": (only "=" or ">" or "<" or "<=" or ">=" whichever is detected else null),
"COMPARISON VALUE": (scalar value, If detected, else null)
}
]
"""
MEASURE_CONSTRAINT = MEASURE_CONSTRAINT+output_format

In [12]:
DERIVED_MEASURE = 

"""Task: Extract the Derived Measure from User Queries

Understand the following Derived Measure definitions:
1. "Ratio": Represents a Derived Measure and can also be referred to as "ratio," "share," "contribution," "percentage," "proportion," or "contributing."
2. "Why": Represents a Derived Measure and has synonyms such as "why," "cause of," "reason for," and "diagnose."
3. "Contribution to Growth": Denotes a Derived Measure and includes synonyms like "contribution to growth," "growth," or "grown."
4. "KDA Transactional": Refers to a Derived Measure with synonyms including "kda," "key drivers," "key driver," "drivers," or "driver."
5. "Growth Rate": Represents a Derived Measure with synonyms like "growth rate," "growth," or "grown."
6. "Correlation": Denotes a Derived Measure and has synonyms such as "associate," "associated," "association," "associations," "correlate," "correlated," "correlation," "correlations," "relate," "related," "relation," "relations," "relationship," or "relationships."

Your task is to examine the user's question to identify the Derived Measure concepts. 
If detected, find the specific targeted derived measure mentioned in the question and return it as the Derived Measure.

Return:
- Derived Measure: (if identified)
- ENTITIES: ["ENTITY1", "ENTITY2"] (if specified in relation to the Derived Measure)
"""

In [18]:
MEASURE = {
        "MEASURE": {
            "{{ entity_from_user_question }}": [
                {
                    "ENTITY": "{{ entity_from_context_data }}",
                    "MEASURE CONSTRAINT":[
                        {
                        "COMPARISON VALUE":"{{ comparison_value }}",
                        "COMPARISON OPERATOR":"{{ comparison_operator }}"
                        }

                    ]
                }
            ]
        }
    }

DIMENSION = {
    "DIMENSION": {
        "{{ entity_from_user_question }}": [
            {
                "ENTITY": "{{ entity_from_context_data }}",
                "ADJECTIVE" : " {{ model_output_adjective }} ",
                "TONE": "{{ model_output_tone }}",
                "RANK": [
                    {
                        "RANK ADJECTIVE": "{{ model_output_rank_adjective }}",
                        "RANK VALUE": "{{ model_output_rank_value }}"
                    }
                ]
            }
        ]
    }
}

FILTER = {
    "FILTER": {
      "{{ entity_from_user_question }}": [
         {
            "ENTITY": "{{ entity_from_context_data }}",
            "EXCLUDE": "{{ exclude_value }}",
            "PARENT": "{{ parent_value }}"
         }
      ]
   }
}

In [19]:
def get_Mql(prompt_type,model_output,context_data): 
    
    if prompt_type=="MEASURE":
        template_content = MEASURE
        template_content = json.dumps(template_content)
        template_content=template_content.replace("{{ entity_from_user_question }}", model_output["ENTITY"])
        template_content=template_content.replace("{{ entity_from_context_data }}", context_data["ENTITY"])
        template_content=template_content.replace("{{ comparison_value }}", model_output["COMPARISON VALUE"])
        template_content=template_content.replace("{{ comparison_operator }}", model_output["COMPARISON OPERATOR"])
        return json.loads(template_content)
    if prompt_type=="DIMENSION":
        template_content = DIMENSION
        template_content = json.dumps(template_content)
        template_content=template_content.replace("{{ entity_from_user_question }}", model_output["ENTITY"])
        template_content=template_content.replace("{{ entity_from_context_data }}", context_data["ENTITY"])
        template_content=template_content.replace("{{ model_output_adjective }}", model_output["ADJECTIVE"])
        template_content=template_content.replace("{{ model_output_tone }}", model_output["TONE"])
        template_content=template_content.replace("{{ model_output_rank_adjective }}", model_output["RANK ADJECTIVE"])
        template_content=template_content.replace("{{ model_output_rank_value }}", model_output["RANK"])
        return json.loads(template_content)
    if prompt_type=="FILTER":
        template_content = FILTER
        template_content = json.dumps(template_content)
        template_content=template_content.replace("{{ entity_from_user_question }}", model_output["ENTITY"])
        template_content=template_content.replace("{{ entity_from_context_data }}", context_data["ENTITY"])
        template_content=template_content.replace("{{ exclude_value }}", str(model_output["EXCLUDE"]))
        template_content=template_content.replace("{{ parent_value }}",str(context_data["parent"]))
        return json.loads(template_content)
    if prompt_type=="DERIVED_MEASURE":
        template_content = DERIVED_MEASURE
        template_content = json.dumps(template_content)
        template_content=template_content.replace("{{ derived_measure_from_user_question }}", model_output["DERIVED MEASURE"])
        template_content=template_content.replace("{{ measure_from_user_question }}", model_output["ENTITY MEASURE"])
        template_content=template_content.replace("{{ measure_from_context }}", context_data["MEASURE"])
        template_content=template_content.replace("{{ comparison_value }}", model_output["COMPARISON VALUE"]) 
        template_content=template_content.replace("{{ comparison_operator }}", model_output["COMPARISON OPERATOR"])
        template_content=template_content.replace("{{ derived_measure_from_context }}", context_data["DERIVED MEASURE"])
        template_content=template_content.replace("{{ filter_list_from_user_question }}", model_output["FILTERS"])
        return json.loads(template_content)
    

In [20]:
context_df = get_context_df()
def get_mql_filter(filter_model_output):
    list_filter_mlqs = []
    for outputs in filter_model_output:
        context_data = find_best_match_from_context(outputs['ENTITY'],context_df)
        Mql= get_Mql(prompt_type="FILTER",model_output=outputs,context_data=context_data)
        list_filter_mlqs.append(Mql)
    filter_mql = {}
    for data in list_filter_mlqs:
        for key, value in data.items():
            if key in filter_mql:
                filter_mql[key].update(value)
            else:
                filter_mql[key] = value
    return filter_mql

In [ ]:
%%time
question = "show me sales greater than 100" 
#question = "in corporate share of phone and binder basis discount"
question = "list of under performing segments and top 5 performing parts in Dubai"

# output_measure=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,MEASURE_CONSTRAINT))
# output_measure_json = json.loads(output_measure.split("[_separator_]")[1])
# print(f"MEASURE_PROMPT output is : {output_measure_json}")
# print("MEASURE_PROMPT Done")

# output_filter = gereneate_output_mistral(compose_prompt_open_ai_prompt(question,FILTER_PROMPT))
# output_filer_json = json.loads("["+f"""{output.split("[_separator_]")[-2]}"""+ "]")
# print(f"FILETER_PROMPT output is : {output_filer_json}")
# print("FILETER_PROMPT Done")

output_dimension=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#print("DIMENSION_PROMPT Done")
#post_process_open_ai_prompt_response(output)
#print(f"""{output.split("[_separator_]")[-2]}""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(output_dimension.split("[_separator_]")[1])

In [ ]:
json.loads(output_measure.split("[_separator_]")[1])

In [91]:
filter_model_output

[{'EXCLUDE': False, 'ENTITY': 'Corporate'},
 {'EXCLUDE': False, 'ENTITY': 'phone'},
 {'EXCLUDE': False, 'ENTITY': 'binder'}]

In [118]:
filter_mql = get_mql_filter(filter_model_output)
filter_mql

{'FILTER': {'Corporate': [{'ENTITY': 'Corporate',
    'EXCLUDE': 'False',
    'PARENT': 'Segment'}],
  'phone': [{'ENTITY': 'Phone', 'EXCLUDE': 'False', 'PARENT': 'Sub-Category'}],
  'binder': [{'ENTITY': 'Binder',
    'EXCLUDE': 'False',
    'PARENT': 'Sub-Category'}]}}

In [14]:
%%time
question = "show me quantity > 100K"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,METRIC_CONSTRAINT))
#post_process_open_ai_prompt_response(output)
print(f"""Metric_constraint model output \n : {output.split("[_separator_]")[1]}""")
try:
    metric_constraint_output = json.loads(output.split("[_separator_]")[1])
except Exception as e:
    print("METRIC_CONSTRAINT model not detected any ENTITIES or Detection was not in a correct format")
    print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Metric_constraint model output 
 : 
            {"COMPARISON OPERATOR": ">",
             "COMPARISON VALUE": "100K",
             "ENTITY": "Quantity"}
            
CPU times: user 7min 22s, sys: 914 ms, total: 7min 23s
Wall time: 1min 4s


In [16]:

context_data = find_best_match_from_context(metric_constraint_output['ENTITY'],context_df)
Mql_for_measure = get_Mql_for_measure(metric_constraint_output,context_data)
Mql_for_measure

{'MEASURE': {'Quantity': [{'ENTITY': 'Quantity',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '100K',
      'COMPARISON OPERATOR': '>'}]}]}}

In [ ]:
%%time
question = "show me the 2 top segments basis sales"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [21]:
%%time
question = "Ratio of sales in Dubai vs india"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,DERIVED_MEASURE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



            Derived Measure: Ratio
            ENTITIES: ["Dubai", "India"]
            
CPU times: user 5min 52s, sys: 947 ms, total: 5min 53s
Wall time: 53.5 s


In [16]:
%%time
question = "sales of Phone in Dubai"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,FILTER_PROMPT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




            **Task: Extract Filter Information**

Identify the filter entities in the user's question and set their inclusion or exclusion . For each filter entity mentioned in the question, check if it's intended to be included or excluded.

Return the following attributes for each detected filter entity:

EXCLUDE: Set to TRUE if the entity is intended to be excluded, FALSE otherwise.
ENTITY: The specific filter entity identified from the question.

If the question specifies exclusion for an entity, set EXCLUDE to TRUE; otherwise, set it to FALSE.

**Return:**
- EXCLUDE: TRUE or FALSE
- ENTITY: The filter entity
            [/ _separator_]

            **Filter Entities:**

            - ENTITY: Phone
            - ENTITY: Dubai

            **Filter Inclusion:**

            - EXCLUDE: FALSE
            - ENTITY: Phone
            - EXCLUDE: FALSE
            - ENTITY: Dubai
CPU times: user 8min 40s, sys: 808 ms, total: 8min 41s
Wall time: 1min 8s


In [17]:
question = "sales of segements except in US"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,FILTER_PROMPT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




            **Task: Extract Filter Information**

Identify the filter entities in the user's question and set their inclusion or exclusion . For each filter entity mentioned in the question, check if it's intended to be included or excluded.

Return the following attributes for each detected filter entity:

EXCLUDE: Set to TRUE if the entity is intended to be excluded, FALSE otherwise.
ENTITY: The specific filter entity identified from the question.

If the question specifies exclusion for an entity, set EXCLUDE to TRUE; otherwise, set it to FALSE.

**Return:**
- EXCLUDE: TRUE or FALSE
- ENTITY: The filter entity
            [/separator_]

            **Filter Entities:**

            - sales
            - segments
            - US

            **Filter Information:**

            - sales: EXCLUDE: FALSE, ENTITY: sales
            - segments: EXCLUDE: FALSE, ENTITY: segments
            - US: EXCLUDE: TRUE, ENTITY: US


In [49]:
%%time
question = "show me quantities > 100K"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,METRIC_CONSTRAINT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




**Task**: Extract "COMPARISON OPERATOR," "COMPARISON VALUE," and the Targeted ENTITY

**Instructions**:

1. Read the input question carefully.
2. Determine if there is a comparison intent present, such as "greater than," "less than," or "equals to." etc, If there is a comparison intent, please proceed to extract the relevant information.
3. Extract the "COMPARISON OPERATOR" based on the detected intent. Examples of "COMPARISON OPERATORS" include "greater than," "less than," and "equals to."
4. Identify and extract the "COMPARISON VALUE" from the sentence. This should be a numerical or scalar value.
5. Detect the ENTITY (e.g., "Sales") on which these comparisons are applied. Please extract the relevant entity.
6. Provide the following information:
   - COMPARISON OPERATOR: (If detected else None) 
   - COMPARISON VALUE: (If detected else None)
   - ENTITY: (If detected else None)

**Input Sentence**: "show me quantities > 100K"

**Output**:
- COMPARISON OPERATOR: greater than
- COMPAR

In [50]:
%%time
question = "When is first time discount of phone is lesser than 0"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,METRIC_CONSTRAINT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




**Task**: Extract "COMPARISON OPERATOR," "COMPARISON VALUE," and the Targeted ENTITY

**Instructions**:

1. Read the input question carefully.
2. Determine if there is a comparison intent present, such as "greater than," "less than," or "equals to." etc, If there is a comparison intent, please proceed to extract the relevant information.
3. Extract the "COMPARISON OPERATORS" based on the detected intent. Examples of "COMPARISON OPERATORS" include "greater than," "less than," and "equals to."
4. Identify and extract the "COMPARISON VALUE" from the sentence. This should be a numerical or scalar value.
5. Detect the ENTITY (e.g., "Sales") on which these comparisons are applied. Please extract the relevant entity.
6. Provide the following information:
   - COMPARISON OPERATOR: (If detected else None) 
   - COMPARISON VALUE: (If detected else None)
   - ENTITY: (If detected else None)

**Input Sentence**: "When is first time discount of phone is lesser than 0"

**Output**:
- COMPARISON OP

In [8]:
%%time
question = "top 2 segments and bottom 3 sub-category basis quantity"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2507: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)




**Task**: Extract Adjectives, Determine Intent (TONE), Identify Entity, Return RANK ADJECTIVE, and Scalar (RANK)

**Instructions**:

1. Carefully read the user question to identify any adjectives present. Adjectives can include terms like "top," "bottom," "best," "worst," etc.
2. Determine the intent (TONE) of the adjective. The intent can be either positive or negative.
   - If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
   - If the adjective intent is positive, always  set RANK ADJECTIVE to "TOP" and TONE to "positive."
3. If the adjective implies a quantity (e.g., "how many," "top 3"), extract that scalar value as RANK.
4. Identify the entity (e.g., "Segment") on which the detected adjective should be applied.

**Input Question**: "top 2 segments and bottom 3 sub-category basis quantity"

**Output**:
- ADJECTIVE: top, bottom
- TONE: mixed (positive and negative)
- ENTITY: Segment, sub-category
- RANK ADJECTIVE: TOP, BOTTOM
- TONE

In [12]:
%%time
question = "show me the 2 top segments basis sales"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2507: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)




**Task**: Extract Adjectives, Determine Intent (TONE), Identify Entity, Return RANK ADJECTIVE, and Scalar (RANK)

**Instructions**:

1. Carefully read the user question to identify any adjectives present. Adjectives can include terms like "least," "highest performing," etc.
2. Determine the intent (TONE) of the adjective. The intent can be either positive or negative.
   - If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
   - If the adjective intent is positive, always  set RANK ADJECTIVE to "TOP" and TONE to "positive."
3. If the adjective implies a quantity (e.g., "how many," "top 3"), extract that scalar value as RANK.
4. Identify the entity (e.g., "Segment") on which the detected adjective should be applied.

**Input Question**: "show me the 2 top segments basis sales"

**Output**:
- ADJECTIVE: top
- TONE: positive
- ENTITY: Segment
- RANK ADJECTIVE: TOP
- TONE: positive
- RANK: 2
CPU times: user 18.8 s, sys: 602 ms, total: 19.4 s

In [14]:
ADJECTIVE_and_TONE_output = post_process_model_output(output)
ADJECTIVE_and_TONE_output

{'ADJECTIVE': 'top',
 'TONE': 'positive',
 'ENTITY': 'Segment',
 'RANK ADJECTIVE': 'TOP',
 'RANK': '2'}

In [12]:
{
    "DIMENSION": {
        "segments": [
            {
                "ENTITY": "Segment",
                "RANK": [
                    {
                        "RANK ADJECTIVE": "top",
                        "RANK VALUE": "2"
                    }
                ]
            }
        ]
    },
    "MEASURE": {
        "sales": [
            {
                "ENTITY": "Sales"
            }
        ]
    }
}


{'DIMENSION': {'segments': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '2'}]}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [16]:
#!pip install fuzzywuzzy

In [11]:
%%time
question = "top 2 and bottom 3 segments by sales"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2507: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 15.60 GiB total capacity; 14.56 GiB already allocated; 14.94 MiB free; 14.91 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF